# Just a simple implementation of a convolutional neural network written in numpy, from the book Grokking Deep Learning

### Foward Propagation:

In [2]:
# Function to select a subregion in a batch of images. It selects the same subregion in a batch of images
def get_image_section(layer, row_from, row_to, col_from, col_to):
    sub_section = layer[:, row_from:row_to, col_from:col_to]
    return sub_section.reshape(-1, 1, row_to-row_from, col_to-col_from)

In [ ]:
layer_0 = images[batch_start:batch_end]
layer_0 = layer_0.reshape(layer_0.shape[0], 28, 28)
layer_0.shape

sects = list()
for row_start in range(layer_0.shape[1] - kernel_rows):
    for col_start in range(layer_0.shape[2] - kernel_cols):
        sect = get_images_section(layer_0,
                                 row_start,
                                 row_start+kernel_rows,
                                 col_start,
                                 col_start+kernel_cols)
        sects.appends(sect)
        
expanded_input = np.concatenate(sects, axis=1)
es = expanded_input.shape
flattened_input = expanded + input.reshape(es[0]*es[1], -1)

In [ ]:
# Randomly initialize the kernels and calculate dot product over the list of subsections
kernels = np.random.random((kernel_rows*kernel_cols, num_kernels))

kernel_output = flattened_input.dot(kernels)

### Full implementation:

In [16]:
import numpy as np, sys
np.random.seed(1)

from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

images, labels = (x_train[0:1000].reshape(1000, 28*28) / 255,
                y_train[0:1000])

one_hot_labels = np.zeros((len(labels), 10))
for i,l in enumerate(labels):
    one_hot_labels[i][l] = 1
labels = one_hot_labels

test_images = x_test.reshape(len(x_test), 28*28) / 255
test_labels = np.zeros((len(y_test), 10))

for i, l in enumerate(y_test):
    test_labels[i][l] = 1
    
def tanh(x):
    return np.tanh(x)

def tanh2deriv(output):
    return 1 - (output ** 2)

def softmax(x):
    temp = np.exp(x)
    return temp / np.sum(temp, axis=1, keepdims = True)

alpha, iterations = (2, 300)
pixels_per_image, num_labels = (784, 10)
batch_size = 128

input_rows = 28
input_cols = 28

kernel_rows = 3
kernel_cols = 3
num_kernels = 16

hidden_size = ((input_rows - kernel_rows) * (input_cols - kernel_cols)) * num_kernels

kernels = 0.2 * np.random.random((kernel_rows * kernel_cols, num_kernels)) - 0.01

weights_1_2 = 0.2 * np.random.random((hidden_size, num_labels))- 0.1

def get_image_section(layer, row_from, row_to, col_from, col_to):
    section = layer[:, row_from:row_to, col_from:col_to]
    return section.reshape(-1, 1, row_to - row_from, col_to - col_from)

for j in range(iterations):
    correct_cnt = 0
    for i in range(int(len(images) / batch_size)):
        batch_start, batch_end = ((i * batch_size), ((i + 1)* batch_size))
        layer_0 = images[batch_start:batch_end]
        layer_0 = layer_0.reshape(layer_0.shape[0], 28, 28)
        layer_0.shape
        
        sects = list()
        for row_start in range(layer_0.shape[1] - kernel_rows):
            for col_start in range(layer_0.shape[2] - kernel_cols):
                sect = get_image_section(layer_0,
                                        row_start,
                                        row_start+kernel_rows,
                                        col_start,
                                        col_start+kernel_cols)
                sects.append(sect)
                
        expanded_input = np.concatenate(sects, axis = 1)
        es = expanded_input.shape
        flattened_input = expanded_input.reshape(es[0] * es[1], -1)
        
        kernel_output = flattened_input.dot(kernels)
        layer_1 = tanh(kernel_output.reshape(es[0], -1))
        dropout_mask = np.random.randint(2, size= layer_1.shape)
        
        layer_1 *= dropout_mask * 2
        layer_2 = softmax(np.dot(layer_1, weights_1_2))
        
        for k in range(batch_size):
            labelset = labels[batch_start+k:batch_start+k+1]
            _inc = int(np.argmax(layer_2[k:k+1]) == np.argmax(labelset))
            correct_cnt += _inc
            
        layer_2_delta = (labels[batch_start:batch_end] - layer_2) / (batch_size * layer_2.shape[0])
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * tanh2deriv(layer_1)
        layer_1_delta *= dropout_mask
        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
        l1d_reshape = layer_1_delta.reshape(kernel_output.shape)
        k_update = flattened_input.T.dot(l1d_reshape)
        kernels -= alpha * k_update
        
    test_correct_cnt = 0
    
    for i in range(len(test_images)):
        
        layer_0 = test_images[i:i+1]
        layer_0 = layer_0.reshape(layer_0.shape[0], 28, 28)
        layer_0.shape
        
        sects = list()
        for row_start in range(layer_0.shape[1] - kernel_rows):
            for col_start in range(layer_0.shape[2] - kernel_cols):
                sect = get_image_section(layer_0,
                                        row_start,
                                        row_start+kernel_rows,
                                        col_start,
                                        col_start+kernel_cols)
                sects.append(sect)
                
        expanded_input = np.concatenate(sects, axis = 1)
        es = expanded_input.shape
        flattened_input = expanded_input.reshape(es[0] * es[1], -1)
        
        kernel_output = flattened_input.dot(kernels)
        layer_1 = tanh(kernel_output.reshape(es[0], -1))
        layer_2 = np.dot(layer_1, weights_1_2)
        
        test_correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[i:i+1]))
        
    if(j % 1 == 0):
        sys.stdout.write("\nI: " + str(j)\
                        + " Test-Acc:" + str(test_correct_cnt/float(len(test_images)))\
                        + " Train-Acc:" + str(correct_cnt/float(len(images))))


I: 0 Test-Acc:0.2425 Train-Acc:0.109
I: 1 Test-Acc:0.4509 Train-Acc:0.183
I: 2 Test-Acc:0.5671 Train-Acc:0.298
I: 3 Test-Acc:0.6439 Train-Acc:0.368
I: 4 Test-Acc:0.6906 Train-Acc:0.4
I: 5 Test-Acc:0.7217 Train-Acc:0.466
I: 6 Test-Acc:0.7429 Train-Acc:0.497
I: 7 Test-Acc:0.7595 Train-Acc:0.51
I: 8 Test-Acc:0.7703 Train-Acc:0.568
I: 9 Test-Acc:0.7807 Train-Acc:0.562
I: 10 Test-Acc:0.788 Train-Acc:0.568
I: 11 Test-Acc:0.7938 Train-Acc:0.61
I: 12 Test-Acc:0.7987 Train-Acc:0.626
I: 13 Test-Acc:0.8041 Train-Acc:0.64
I: 14 Test-Acc:0.8059 Train-Acc:0.621
I: 15 Test-Acc:0.8094 Train-Acc:0.645
I: 16 Test-Acc:0.8154 Train-Acc:0.64
I: 17 Test-Acc:0.8196 Train-Acc:0.664
I: 18 Test-Acc:0.821 Train-Acc:0.676
I: 19 Test-Acc:0.8246 Train-Acc:0.653
I: 20 Test-Acc:0.8252 Train-Acc:0.68
I: 21 Test-Acc:0.8288 Train-Acc:0.684
I: 22 Test-Acc:0.8295 Train-Acc:0.686
I: 23 Test-Acc:0.8313 Train-Acc:0.666
I: 24 Test-Acc:0.8339 Train-Acc:0.687
I: 25 Test-Acc:0.8336 Train-Acc:0.694
I: 26 Test-Acc:0.8353 Train-Ac

I: 214 Test-Acc:0.8733 Train-Acc:0.812
I: 215 Test-Acc:0.8741 Train-Acc:0.81
I: 216 Test-Acc:0.8739 Train-Acc:0.8
I: 217 Test-Acc:0.8723 Train-Acc:0.809
I: 218 Test-Acc:0.8719 Train-Acc:0.82
I: 219 Test-Acc:0.874 Train-Acc:0.821
I: 220 Test-Acc:0.8734 Train-Acc:0.81
I: 221 Test-Acc:0.8724 Train-Acc:0.805
I: 222 Test-Acc:0.8721 Train-Acc:0.807
I: 223 Test-Acc:0.8722 Train-Acc:0.797
I: 224 Test-Acc:0.8716 Train-Acc:0.813
I: 225 Test-Acc:0.8722 Train-Acc:0.813
I: 226 Test-Acc:0.8741 Train-Acc:0.803
I: 227 Test-Acc:0.8743 Train-Acc:0.812
I: 228 Test-Acc:0.8745 Train-Acc:0.804
I: 229 Test-Acc:0.8743 Train-Acc:0.817
I: 230 Test-Acc:0.8718 Train-Acc:0.802
I: 231 Test-Acc:0.8725 Train-Acc:0.817
I: 232 Test-Acc:0.8721 Train-Acc:0.819
I: 233 Test-Acc:0.8729 Train-Acc:0.809
I: 234 Test-Acc:0.8722 Train-Acc:0.819
I: 235 Test-Acc:0.8722 Train-Acc:0.812
I: 236 Test-Acc:0.873 Train-Acc:0.805
I: 237 Test-Acc:0.8729 Train-Acc:0.81
I: 238 Test-Acc:0.8718 Train-Acc:0.816
I: 239 Test-Acc:0.8711 Train-Acc: